In [1]:
pip install langchain-core langgraph>0.2.27

Note: you may need to restart the kernel to use updated packages.


In [13]:
 pip install langchain openai langchain_openai faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [ ]:
 %restart_python

In [10]:
import os

#Setting Environment variable
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://dskumar.openai.azure.com/'
os.environ["AZURE_OPENAI_API_KEY"] ="62855d6dd08945819bf83aee0c104127"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] ="DskumarDeployment"
os.environ['OPENAI_TYPE']="Azure"
os.environ["LLM_MODEL"] = "gpt-35-turbo-16k"
os.environ["LLM_EMBEDDING_MODEL"] = "dskumar-text-embedding-ada-002"

In [5]:
 endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
 key = os.environ["AZURE_OPENAI_API_KEY"]

In [6]:
 from langchain_core.documents import Document

 documents = [
      Document(page_content="Azure Databricks is a fast, easy, and collaborative Apache Spark-based analytics platform.", metadata={"date_created": "2024-08-22"}),
      Document(page_content="LangChain is a framework designed to simplify the creation of applications using large language models.", metadata={"date_created": "2024-08-22"}),
      Document(page_content="GPT-4 is a powerful language model developed by OpenAI.", metadata={"date_created": "2024-08-22"})
 ]
 ids = ["1", "2", "3"]

In [11]:
 from langchain_openai import AzureOpenAIEmbeddings
     
 embedding_function = AzureOpenAIEmbeddings(
     deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"], 
     model=os.environ["LLM_EMBEDDING_MODEL"],
     azure_endpoint=endpoint,
     openai_api_key=key,
     chunk_size=1
 )

In [14]:
 import faiss
      
 index = faiss.IndexFlatL2(len(embedding_function.embed_query("Azure Databricks is a fast, easy, and collaborative Apache Spark-based analytics platform.")))

BadRequestError: Error code: 400 - {'error': {'code': 'OperationNotSupported', 'message': 'The embeddings operation does not work with the specified model, gpt-35-turbo-16k. Please choose different model and try again. You can learn more about which models can be used with each operation here: https://go.microsoft.com/fwlink/?linkid=2197993.'}}

In [ ]:
 from langchain.vectorstores import FAISS
 from langchain_core.vectorstores import VectorStoreRetriever
 from langchain_community.docstore.in_memory import InMemoryDocstore

 vector_store = FAISS(
     embedding_function=embedding_function,
     index=index,
     docstore=InMemoryDocstore(),
     index_to_docstore_id={}
 )
 vector_store.add_documents(documents=documents, ids=ids)
 retriever = VectorStoreRetriever(vectorstore=vector_store)

In [ ]:
 from langchain_openai import AzureChatOpenAI
 from langchain_core.prompts import ChatPromptTemplate
 from langchain.chains.combine_documents import create_stuff_documents_chain
 from langchain.chains import create_retrieval_chain
     
 llm = AzureChatOpenAI(
     deployment_name="gpt-35-turbo-16k",
     model_name="gpt-35-turbo-16k",
     azure_endpoint=endpoint,
     api_version="2023-03-15-preview",
     openai_api_key=key,
 )

 system_prompt = (
     "Use the given context to answer the question. "
     "If you don't know the answer, say you don't know. "
     "Use three sentences maximum and keep the answer concise. "
     "Context: {context}"
 )

 prompt1 = ChatPromptTemplate.from_messages([
     ("system", system_prompt),
     ("human", "{input}")
 ])

 chain = create_stuff_documents_chain(llm, prompt)

 qa_chain1 = create_retrieval_chain(retriever, chain)

In [ ]:
 result = qa_chain1.invoke({"input": "What is Azure Databricks?"})
 print(result)